### Scrapping todays & yesturdays article on multiple pages of the Champion News website by searching for the  country-name(or any keyword).

In [1]:
# importing modules
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date, timedelta, datetime
import time
import re

In [2]:
# Set up the web driver (provide the path to your Firefox web driver)
chrome_options = webdriver.FirefoxOptions()

chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--verbose")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

driver = webdriver.Firefox(options=chrome_options)

driver.get("https://championnews.com.ng")

In [3]:
# Clicking the search icon
search_icon = driver.find_element(By.XPATH, "//span[@class='search-handler']")
search_icon.click()
# Get the input elements
search_input = driver.find_element(By.XPATH, "//input[@class='search-field']")
search_button = driver.find_element(By.XPATH, "//input[@value='Search']")

# Send the input to the webpage
search_input.send_keys('nigeria')
search_input.send_keys(Keys.RETURN)

In [4]:
# Define today's date and yesterday's date

today = date.today()
yesterday = today - timedelta(days=1)

def post_date_check(post_date):
    date_match = re.search(r'(\w{3})\s(\d{1,2}),\s(\d{4})', post_date)
    
    if date_match:
        month, day, year = date_match.groups()
        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }
        month_number = month_map.get(month)
        article_date = datetime(int(year), month_number, int(day)).date()

        return article_date == today or article_date == yesterday

    return False


In [6]:
print(post_date_check('Oct 29, 2023'))
print(post_date_check('Oct 27, 2023'))

True
False


In [7]:
# list for storing the articles url
todays_and_yesturdays_article_urls = []
# list for storing the articles post-date becouse post-date not available in single article page
articles_postdate = []

# Scrapping articles with pagination
new_articles = True

# Loop through the pages
while new_articles:
    
    # Scrolling down to load more search results
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
    
    # collecting all articles from the current page
    article_list = driver.find_elements(By.XPATH, "//div//article")
    
    # Extract the URLs of today's and yesterday's articles 
    for article in article_list:
        # Extract the date
        date_text = article.find_element(By.CLASS_NAME, "time").text
        date_match = post_date_check(date_text)

        if date_match:
            a_link = article.find_element(By.TAG_NAME, "h2").find_element(By.TAG_NAME, "a").get_attribute("href")
            todays_and_yesturdays_article_urls.append(a_link)
            articles_postdate.append(date_text)
        else:
            new_articles = False
            # break
        
    # Find and click the pagination link for the next page
    try:
        next_page_link = driver.find_element(By.XPATH, "//div[@class='older']//a")
        next_page_link.click()
    except NoSuchElementException:
        print("No 'Next Page' link found. Exiting the loop.")
        break  # Exit the loop if there is no "Next Page" link

In [8]:
len(todays_and_yesturdays_article_urls)

4

In [9]:
len(articles_postdate)

4

In [15]:
# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Initialize an empty list to store dictionaries
data_list = []

for article_url in todays_and_yesturdays_article_urls:
    driver.get(article_url)
    # Wait for the article page to load
    wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
    article_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h1[@class='single-post-title']")))
    
    # collecting the data
    headline = driver.find_element(By.XPATH, "//h1[@class='single-post-title']").text
    p_content = driver.find_elements(By.XPATH, "//div[@class='continue-reading-content close']//p")
    content = ''
    for i in p_content:
        content += i.text
        
    content = content.replace("    ", " ")
    
    # Append the data to the list as a dictionary
    data_list.append({"Headline": headline, "Content": content})
    

In [16]:
# Create the DataFrame by concatenating the list of dictionaries
df = pd.DataFrame(data_list)

In [17]:
df['Post_date'] = articles_postdate

In [18]:
len(df)

4

In [19]:
df.sample(1)

Headline  \
1  Idris urges Nigerians to unite for the Renewed Hope Agenda as the Supreme Court validates Presidential Election Result   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
# storing data in to excel file
df.to_excel('championnews.xlsx', index=False)

In [20]:
driver.quit()